In [ ]:
# Logist regression .Initating the scaler 
scaler = StandardScaler()

# Fitting the scaler
Xss=scaler.fit_transform(X)

logreg = LogisticRegression(C=10**10,solver='lbfgs')
logreg.fit(Xss, high_low)

print('Logreg intercept:', logreg.intercept_)
print('Logreg coef(s):', logreg.coef_)
print('Logreg predicted probabilities:\n', logreg.predict_proba(X.iloc[0:5,:]))
print('Logreg predicted classes:', logreg.predict(X.iloc[0:5,:]))

from sklearn.model_selection import cross_val_score

scores = cross_val_score(logreg,Xss, high_low,cv=10)
print(scores)
print(np.mean(scores))

# The accuracy of the model
np.mean(scores)

print('Logreg predicted labels:\n',logreg.predict(X)[:500])

# Putting the coefs into a df and sorting 
coef_df = pd.DataFrame(data=logreg.coef_[0],index=X.columns)
coef_df['abs'] = np.abs(coef_df[0])
coef_df.sort_values(by="abs",ascending=False,inplace=True)

# Viewing the coefs in order of impact 
fig, ax = plt.subplots(figsize=(20,20))
coef_df['abs'].plot(kind="barh",ax=ax)
plt.show()

# Viewing coefs in order of importance and how the effect the model.
fig, ax = plt.subplots(figsize=(20,20))
coef_df[0].plot(kind="barh",ax=ax)
plt.show()

coef_df

from sklearn import metrics

logreg.predict(Xss).shape

# Viewing the confusion matrix 
confusion_matrix = metrics.confusion_matrix(high_low, logreg.predict(Xss))
print(metrics.classification_report(high_low, logreg.predict(Xss)))

# viewing the precion scores 
precision_scores = metrics.precision_score(high_low,logreg.predict(Xss), average=None)
print(precision_scores)
print("Macro:",metrics.precision_score(high_low,logreg.predict(Xss), average='macro'))
print("Micro:",metrics.precision_score(high_low,logreg.predict(Xss), average='micro'))

In [ ]:
# Plotting precion-recall curve
y_bin = label_binarize(high_low, logreg.classes_)
y_bin[:10]

def plot_precision_recall(model, y_bin, X, f1_lines=True):
    '''
    Takes a fitted model, 
    an array of binarised true y-values, 
    a predictor matrix and
    and additional argument if lines of constant f1-score levels should be plotted.
    Returns a precision-recall plot for each of the classes.
    '''

    # create dictionaries for storing precision/recall values
    precision = {}
    recall = {}

    # loop over all the classes
    # calculate precision-recall values for the current class versus the rest
    for i, cl in enumerate(model.classes_):
        precision[cl], recall[cl], threshold = metrics.precision_recall_curve(
            y_bin[:, i], model.predict_proba(X)[:, i])

    # average_precision_score calculates the area under the curve for each class
    average_precision_scores = [round(metrics.average_precision_score(
        y_bin[:, i], model.predict_proba(X)[:, i]), 2) for i in range(len(model.classes_))]

    # plot the precision-recall curves for each class versus the rest
    fig, ax = plt.subplots(figsize=(6,4))
    for i, key in enumerate(precision.keys()):
        ax.plot(precision[key], recall[key], lw=2,
                 label='class {}, average precision score {}'.format(key, average_precision_scores[i]))

    ax.set_ylim([0, 1.05])
    ax.legend(loc=[1,0])
    ax.set_title('Precision-Recall Curve', fontsize=20)
    ax.set_xlabel('Recall', fontsize=18)
    ax.set_ylabel('Precision', fontsize=18)

    # add lines of constant F1 scores
    if f1_lines == True:
        for const in np.linspace(0.2,0.9,8):
            x_vals = np.linspace(0.001, 0.999, 100)
            y_vals = 1./(2./const-1./x_vals)
            ax.plot(x_vals[y_vals > 0], y_vals[y_vals > 0],
                     color='lightblue', ls='--', alpha=0.9)
            ax.set_ylim([0, 1])
            ax.annotate('f1={0:0.1f}'.format(const),
                         xy=(x_vals[-10], y_vals[-2]+0.0))

    return fig, ax